<a href="https://colab.research.google.com/github/tomasborrella/TheValley/blob/main/notebooks/spark01/Ejercicio_resuelto_MapReduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio resuelto de MapReduce con Python

Notebook por [Tomás Borrella Martín](https://www.linkedin.com/in/tomasborrella/).

### Enlaces de interés
*   [Slides de presentación](https://docs.google.com/presentation/d/10HZGQnFNzRO63I9XRt-uQa6K9K2yAM71Wu-SYB0TL7c/edit?usp=sharing)

## 1. Datos
Descargamos un archivo que contiene una canción en cada fila (simplificado para el ejemplo).

NOTA: En un notebook, "!" ejecuta comandos del sistema desde dentro del notebook.

In [1]:
!wget -P /content/data 'https://raw.githubusercontent.com/tomasborrella/TheValley/main/data/spark01/complete_songs_log.txt' 

--2021-06-05 09:41:38--  https://raw.githubusercontent.com/tomasborrella/TheValley/main/data/spark01/complete_songs_log.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 126125 (123K) [text/plain]
Saving to: ‘/content/data/complete_songs_log.txt’

complete_songs_log. 100%[===================>] 123.17K  --.-KB/s    in 0.04s   

2021-06-05 09:41:38 (3.16 MB/s) - ‘/content/data/complete_songs_log.txt’ saved [126125/126125]



Mostramos el contenido del archivo

In [4]:
# Las primeras 5 líneas
! head -n 5 /content/data/complete_songs_log.txt

Blinding Lights, The Weeknd, 1590851957, Tomas, Spain
Dance Monkey, Tones and I, 1590851960, John, USA
Blinding Lights, The Weeknd, 1590851957, Tomas, Spain
Dance Monkey, Tones and I, 1590851960, John, USA
The Box, Roddy Ricch, 1590851961, Sarah, Canada


In [5]:
# Las últimas 5 líneas
! tail -n 5 /content/data/complete_songs_log.txt

Blinding Lights, The Weeknd, 1590851957, Tomas, Spain
Blinding Lights, The Weeknd, 1590851957, Tomas, Spain
Dance Monkey, Tones and I, 1590851960, John, USA
Blinding Lights, The Weeknd, 1590851957, Tomas, Spain
The Box, Roddy Ricch, 1590851961, Sarah, Canada

In [6]:
# Vemos cuantas líneas tiene en total
! wc -l /content/data/complete_songs_log.txt

2468 /content/data/complete_songs_log.txt


## 2. Instalamos la librería que vamos a usar para el ejemplo
mrjob es un paquete que permite ejecutar MapReduce jobs con Python

[Documentación oficial](https://mrjob.readthedocs.io/en/latest/index.html#)

In [7]:
! pip install mrjob

     |████████████████████████████████| 440kB 3.9MB/s 


## 3. Creamos el archivo del programa

In [8]:
%%file wordcount_final.py
# %%file: "magic function" que guarda el contenido de una celda a un archivo

# Importamos la librería mrjob
from mrjob.job import MRJob 

# Creamos una clase que hereda de MRJob
class MRSongCount(MRJob):
    
    # En la fase de Map cada linea del archivo de texto se lee como una pareja 
    # clave valor (key, value) que son los parámetros 2 y 3 del "mapper".
    # En este caso cada linea no contiene ninguna key sino solo el valor, 
    # por eso usamos _ como segundo parámetro (campo key).

    def mapper(self, _, line):
        line_splited = line.split(", ")
        song = line_splited[0]
        # La salida es una dupla (nombre_cancion, 1) 
        yield (song, 1)

    # En la fase de Reduce se combina cada dupla con la misma clave.
    # (en el ejemplo la clave es el nombre de la canción)
    # En este caso la forma de combinarlo es sumando, lo que nos dará
    # el total de reproducciones de cada canción
    
    def reducer(self, key, values):
        # La salida es una dupla (nombre_cancion, suma_de_reproducciones)
        yield (key, sum(values))
        
if __name__ == "__main__":
    MRSongCount.run()

Writing wordcount_final.py


## 4. Ejecutamos el programa

In [9]:
# run the code as a terminal command
! python wordcount_final.py /content/data/complete_songs_log.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/wordcount_final.root.20210605.094501.479240
Running step 1 of 1...
job output is in /tmp/wordcount_final.root.20210605.094501.479240/output
Streaming final output from /tmp/wordcount_final.root.20210605.094501.479240/output...
"Blinding Lights"	1131
"The Box"	510
"Dance Monkey"	828
Removing temp directory /tmp/wordcount_final.root.20210605.094501.479240...
